In [1]:
%pip install pandas 

import pandas as pd
import sqlite3
import jieba

Note: you may need to restart the kernel to use updated packages.


In [2]:
data = pd.read_csv("./data/wukong50k_release.csv")

In [21]:
def check_contain_chinese(check_str):
    for ch in check_str:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
        if "a" <= ch <= "z" or "A" <= ch <= "X":
            return True
        if "0" <= ch <= "9":
            return True
    return False

In [22]:
data2 = []
for d in data.itertuples():
    title = d[2]
    url = d[1]
    cut = jieba.cut(title)
    keyword = ""
    for c in cut:
        if check_contain_chinese(c):
            keyword += " " + c
    keyword = keyword.strip()  
    data2.append([title, keyword, url])

In [ ]:
data2

In [ ]:
data3 = pd.DataFrame(data2, columns=["title", "keyword", "url"])
data3

In [25]:
data3.to_csv("./data/cleaned_database.csv", index=False)

In [26]:
for line in data3.itertuples():
    title, keyword, url = line[1],line[2],line[3]
    print(title)
    print(keyword)
    print(url)
    break
    

美沃可视数码裂隙灯,检查眼前节健康状况
美沃 可视 数码 裂隙 灯 检查 眼前 节 健康状况
https://gimg2.baidu.com/image_search/src=http%3A%2F%2F5b0988e595225.cdn.sohucs.com%2Fimages%2F20200326%2Fffc00cb6bc944e5b9ab2673c4873b24c.jpeg&refer=http%3A%2F%2F5b0988e595225.cdn.sohucs.com&app=2002&size=f9999,10000&q=a80&n=0&g=0n&fmt=jpeg?sec=1632531279&t=1b9ba84f70ddebdda6601a5576d37c50


In [30]:
conn = sqlite3.connect("./data/database.db")
c = conn.cursor()

# 创建数据库
sql = "drop table page_info;"
c.execute(sql)
conn.commit()

sql = "drop table page_index;"
c.execute(sql)
conn.commit()

sql = """
    create table page_info(
        id INTEGER PRIMARY KEY,
        keyword text not null,
        url text not null
    );
"""
c.execute(sql)
conn.commit()


# 创建索引表
sql = """
    create table page_index(
        id INTEGER PRIMARY KEY,
        keyword text not null,
        page_id INTEGER not null
    );
"""
c.execute(sql)
conn.commit()

In [32]:



# 插入到数据库
i = 0
for line in data3.itertuples():
    title, keyword, url = line[1],line[2],line[3]
    sql = """
        insert into page_info (url, keyword) 
        values('%s', '%s')
    """ % (url, keyword)
    c.execute(sql)
    conn.commit()
    i += 1
    if i % 50 == 0:
        print(i, len(data3))
        
        

sql = "delete from page_index;"
c.execute(sql)
conn.commit()

sql = "select * from page_info;"
res = c.execute(sql)
res = list(res)
length = len(res)

i = 0
for line in res:
    pid, words, url = line[0], line[1], line[2]
    words = words.split(" ")
    for w in words:
        sql = """
        insert into page_index (keyword, page_id) 
        values('%s', '%s')
        """ % (w, pid)
        c.execute(sql)
        conn.commit()
    i += 1
    if i % 100 == 0:
        print(i, length)

50 50258
100 50258
150 50258
200 50258
250 50258
300 50258
350 50258
400 50258
450 50258
500 50258
550 50258
600 50258
650 50258
700 50258
750 50258
800 50258
850 50258
900 50258
950 50258


KeyboardInterrupt: 

In [45]:
titles = list(words)
colums = ["title", "url"] + titles
word_vector = pd.DataFrame(columns=colums)
word_vector

,title,url,Momo,徐风,可靠消息,伴君,➇,大家闺秀,HZQ,魂淡,...,160426,浪人,miniso,懒懒,被迫,雷,一言难尽,看眼南,盖楼,charging


In [2]:
data = pd.read_csv("./data/database.csv")

In [ ]:
data

In [3]:
sql = "alter table page_info add title text;"
conn = sqlite3.connect("./data/database.db")
c = conn.cursor()
c.execute(sql)
conn.commit()

OperationalError: duplicate column name: title

In [8]:
conn = sqlite3.connect("./data/database.db")
c = conn.cursor()
length = len(data)
i = 0
for line in data.itertuples():

    pid = line[0]+1
    title = line[2]
    # print(pid, title)
    
    sql = "UPDATE page_info SET title = '%s' WHERE id = %s "%(title,pid)
    try:
        c.execute(sql)
        conn.commit()
    except:
        continue
    i += 1
    if i % 50 == 0:
        print(i, length)



50 50258
100 50258
150 50258
200 50258
250 50258
300 50258
350 50258
400 50258
450 50258
500 50258
550 50258
600 50258
650 50258
700 50258
750 50258
800 50258
850 50258
900 50258
950 50258
1000 50258
1050 50258
1100 50258
1150 50258
1200 50258
1250 50258
1300 50258
1350 50258
1400 50258
1450 50258
1500 50258
1550 50258
1600 50258
1650 50258
1700 50258
1750 50258
1800 50258
1850 50258
1900 50258
1950 50258
2000 50258
2050 50258
2100 50258
2150 50258
2200 50258
2250 50258
2300 50258
2350 50258
2400 50258
2450 50258
2500 50258
2550 50258
2600 50258
2650 50258
2700 50258
2750 50258
2800 50258
2850 50258
2900 50258
2950 50258
3000 50258
3050 50258
3100 50258
3150 50258
3200 50258
3250 50258
3300 50258
3350 50258
3400 50258
3450 50258
3500 50258
3550 50258
3600 50258
3650 50258
3700 50258
3750 50258
3800 50258
3850 50258
3900 50258
3950 50258
4000 50258
4050 50258
4100 50258
4150 50258
4200 50258
4250 50258
4300 50258
4350 50258
4400 50258
4450 50258
4500 50258
4550 50258
4600 50258
4650 502